#**Implementation of Random Forest and Hyper Parameter Tuning on pre-processed Lending Club Dataset:**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/Data"

 accepted_2007_to_2018Q4.csv  "Lending_Club's_loan_approval_optimization.csv"
 LCDataDictionary.xlsx	       rejected_2007_to_2018Q4.csv


In [5]:
%cd /content/drive/My Drive/Data/

/content/drive/My Drive/Data


In [6]:
#importing python libraries for analysis:

import pandas as pd
import numpy as np

In [8]:
df_Lending_Club = pd.read_csv("Lending_Club's_loan_approval_optimization.csv")
df_Lending_Club.drop("Unnamed: 0", axis=1, inplace=True)
df_Lending_Club.head()

,Amount Requested,Risk_Score,Debt-To-Income Ratio,Employment Length,Target
0,3600.0,677.0,5.91,10,1
1,24700.0,717.0,16.06,10,1
2,20000.0,697.0,10.78,10,1
3,10400.0,697.0,25.37,3,1
4,11950.0,692.0,10.20,4,1


#**Let's split the data for training and testing:**



In [9]:
from sklearn.model_selection import train_test_split,cross_val_predict

X = df_Lending_Club.iloc[:,:-1].values
y = df_Lending_Club.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
print(X_train.shape)
print(y_train.shape)

(1658244, 4)
(1658244,)


In [11]:
from sklearn.ensemble import RandomForestClassifier

Model_RF = RandomForestClassifier()
Model_RF.fit(X_train, y_train)
y_pred = Model_RF.predict(X_test)
print("Accuracy score of training data {} %:".format(Model_RF.score(X_train, y_train)*100))
print("Accuracy score of testing data {} %".format(Model_RF.score(X_test, y_test)*100))

Accuracy score of training data 99.33562250187548 %:
Accuracy score of testing data 85.78765058061279 %


#**Let's Retrain the Model by using the best parameters which is discovered from RandomizedSearchCV:**

stratified : n_estimators=500, max_features=None, max_depth=6, bootstrap=True

In [21]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=500, max_features=None, max_depth=6, bootstrap=True)
model.fit(X_train, y_train)
print("Accuracy score of training data {} %:".format(model.score(X_train, y_train)*100))
print("Accuracy score of testing data {} %".format(model.score(X_test, y_test)*100))

Accuracy score of training data 88.80056252276505 %:
Accuracy score of testing data 88.80577573438954 %


In [22]:
# AUC Score
from sklearn.metrics import roc_auc_score, mean_absolute_error

auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print("\nAUC SCORE: {:.3f}".format(auc))


AUC SCORE: 0.917


In [23]:
y_pred = model.predict([[36000, 750, 25, 9]])
y_pred

array([1])

In [24]:
y_pred = model.predict([[20000, 550, 25, 6]])
y_pred

array([0])

<h1>Deployment</h1>

In [25]:
import pickle

In [26]:
pickle.dump(model, open('lending_loan.pkl', 'wb'))

In [27]:
with open('lending_loan.pkl', 'rb') as f:
    auc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print("\nAUC SCORE: {:.3f}".format(auc_score))


AUC SCORE: 0.917
